# `snopy` - basic usage and data querying

## Imports

In [1]:
import os
from snopy import snopy_connect
from snowflake.connector.errors import ProgrammingError

## Credentials

In [2]:
sf_username=os.environ['SNOWFLAKE_USER']
sf_password=os.environ['SNOWFLAKE_PASSWORD']
sf_account=os.environ['SNOWFLAKE_ACCOUNT']

## Connecting to your Snowflake account

In [3]:
sc = snopy_connect(
    username=sf_username,
    password=sf_password,
    account=sf_account
)

## Environment setup

In [4]:
sc.database.get_current() is None

True

In [5]:
sc.database.use('SNOWFLAKE_SAMPLE_DATA', silent=True)

In [6]:
sc.role.get_current()

'ACCOUNTADMIN'

In [7]:
sc.set_environment(
    warehouse='COMPUTE_WH',
    schema='TPCH_SF100'
)

In [8]:
sc.get_environment()

{'role': 'ACCOUNTADMIN',
 'database': 'SNOWFLAKE_SAMPLE_DATA',
 'schema': 'TPCH_SF100',
 'warehouse': 'COMPUTE_WH'}

## Creating new database and schema

In [9]:
sc.database.create('SNOPY', or_replace=True)

{'results': [('Database SNOPY successfully created.',)],
 'description': [ResultMetadata(name='status', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True)],
 'statement': 'CREATE OR REPLACE DATABASE SNOPY'}

In [10]:
sc.database.use('SNOPY', silent=True)

In [11]:
sc.schema.create('INGESTION_SCHEMA', silent=True)

In [12]:
sc.schema.use('INGESTION_SCHEMA', True)

## Executing custom query with a cursor

In [13]:
sc.cursor.execute('SHOW SCHEMAS;').fetchall()

[(datetime.datetime(2022, 9, 15, 8, 30, 4, 889000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'INFORMATION_SCHEMA',
  'N',
  'N',
  'SNOPY',
  '',
  'Views describing the contents of schemas in this database',
  '',
  '1'),
 (datetime.datetime(2022, 9, 15, 8, 30, 1, 216000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'INGESTION_SCHEMA',
  'N',
  'Y',
  'SNOPY',
  'ACCOUNTADMIN',
  '',
  '',
  '1'),
 (datetime.datetime(2022, 9, 15, 8, 30, 0, 596000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'PUBLIC',
  'N',
  'N',
  'SNOPY',
  'ACCOUNTADMIN',
  '',
  '',
  '1')]

## Data querying - wrong schema

In [14]:
query = 'SELECT * FROM CUSTOMER;'

In [15]:
# Querying table that doesn't exist in SNOPY/INGESTION_SCHEMA
try:
    results_1_not_working = sc.execute(query)
except ProgrammingError as pe:
    print('Something went wrong!')
    print(pe)

Something went wrong!
002003 (42S02): SQL compilation error:
Object 'CUSTOMER' does not exist or not authorized.


## Data querying - valid command run with session environment properly set up

In [16]:
sc.set_environment(
    database = 'SNOWFLAKE_SAMPLE_DATA',
    schema = 'TPCH_SF100'
)

In [17]:
results_1_working = sc.execute(query)
results_1_working

{'results': [(5450001,
   'Customer#005450001',
   'j 655pYx,ZbJ0Ktq4sB Ef8lFfD0Di',
   20,
   '30-681-597-5795',
   Decimal('3503.51'),
   'MACHINERY',
   'the slyly even theodolites cajole final platelets: sly'),
  (5450002,
   'Customer#005450002',
   'JGNavHUQ3bVxX,tjYDL3H',
   18,
   '28-481-591-7459',
   Decimal('9540.18'),
   'HOUSEHOLD',
   'd sheaves. regular braids solve after the carefully pending requests'),
  (5450003,
   'Customer#005450003',
   'RWDNNP6Wrg7wTZk1RRCdhpxOtVQ',
   7,
   '17-990-536-2087',
   Decimal('3886.61'),
   'BUILDING',
   'eposits. quickly special grouches'),
  (5450004,
   'Customer#005450004',
   'rgUN9YK3qksvye7BYG3WJTUd5s,rs6,rJVl4CQTL',
   5,
   '15-634-829-4041',
   Decimal('2282.72'),
   'AUTOMOBILE',
   'ses wake alongside of the express foxes. r'),
  (5450005,
   'Customer#005450005',
   'EMuO7IiXmwdzgODlWb,HOw8l5JTqJLply',
   15,
   '25-925-600-1012',
   Decimal('6620.67'),
   'HOUSEHOLD',
   'the furiously regular requests. '),
  (5450006,

## Data querying - another valid command run

In [18]:
results_2 = sc.execute(query, n=100)

print(f"Rows returned: {len(results_2['results'])}")
results_2['results'][:2]

Rows returned: 100


[(5450001,
  'Customer#005450001',
  'j 655pYx,ZbJ0Ktq4sB Ef8lFfD0Di',
  20,
  '30-681-597-5795',
  Decimal('3503.51'),
  'MACHINERY',
  'the slyly even theodolites cajole final platelets: sly'),
 (5450002,
  'Customer#005450002',
  'JGNavHUQ3bVxX,tjYDL3H',
  18,
  '28-481-591-7459',
  Decimal('9540.18'),
  'HOUSEHOLD',
  'd sheaves. regular braids solve after the carefully pending requests')]

## Data querying - mapping results to Pandas DataFrame

In [19]:
results_pandas = sc.query_pd(query, n=100)

print(f'Rows returned: {len(results_pandas)}')
results_pandas.head(2)

Rows returned: 100


,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,5450001,Customer#005450001,"j 655pYx,ZbJ0Ktq4sB Ef8lFfD0Di",20,30-681-597-5795,3503.51,MACHINERY,the slyly even theodolites cajole final platel...
1,5450002,Customer#005450002,"JGNavHUQ3bVxX,tjYDL3H",18,28-481-591-7459,9540.18,HOUSEHOLD,d sheaves. regular braids solve after the care...


# Extra

### Objects: `cursor` and `connection`

Both provided by Snowflake connector, that allow to execute any functionality possible with the original connector.

In [20]:
sc.cursor.execute('SELECT CURRENT_WAREHOUSE()').fetchall()

[('COMPUTE_WH',)]

In [21]:
sc.connection.database

'SNOWFLAKE_SAMPLE_DATA'

## Closing the connection

In [22]:
sc.close_connection()